## Module 4 Practice 1 - DICOM data extraction

In this exercise, you will extract the metadata from a directory containing multiple subjects, and perform basic descriptive analysis of that data.

The directory is the same as used in the lab, except we will read the DICOM files of every patient represented rather than just one.

This is the structure:

```
Brain-Tumor-Progression
  |_ PGBM-001 (the patient identifier in the study)
    |_ 11-19-1991-FH-HEADBrain Protocols-40993 (folder for the scan date)
        |_ 11.000000-T1post-03326 (folder for the scan series)
            |_ 1-01.dcm (sequential scan file)
            ...
            |_ 1-25.dcm (sequential scan file)
    |_ 04-02-1992-FH-HEADBrain Protocols-79896 (folder for the scan date)
        |_ 11.000000-T1post-80644 (folder for the scan series)
            |_ 1-01.dcm (sequential scan file)
            ...
            |_ 1-24.dcm (sequential scan file)
  |_ PGBM-002
    |_ <scan date 1>
        |_ <series>
            |_ 1-01.dcm
            ...
            |_ 1-xx.dcm (the number of images in each series varies)
    |_ <scan date 2>
        |_ <series>
            |_ 1-01.dcm
            ...
            |_ 1-xx.dcm (the number of images in each series varies)
  ...
  |_ PGBM-005 (there are 5 patients)
    |_ <scan date 1>
        |_ <series>
            |_ 1-01.dcm
            ...
            |_ 1-xx.dcm (the number of images in each series varies)
    |_ <scan date 2>
        |_ <series>
            |_ 1-01.dcm
            ...
            |_ 1-xx.dcm (the number of images in each series varies)
  
```

There are five subjects, two scan dates per subject, and possibly multiple series for each scan date.

In [1]:
import sys
!{sys.executable} -m pip install pydicom scikit-image
import pydicom
import skimage
import numpy as np
import pandas as pd

     |████████████████████████████████| 2.0MB 3.3MB/s eta 0:00:01


## Extract data from all DICOM files

Loop over all dicom images under the `Brain-Tumor-Progression` directory structure and extract the following tags into a pandas dataframe.  Refer to the lab on recursive directory looping for the technique to read all of the images.

```
(0010, 0010) Patient's Name                      
(0010, 0020) Patient ID                          
(0010, 0030) Patient's Birth Date                
(0010, 0040) Patient's Sex                       
(0010, 1010) Patient's Age                       
(0010, 1030) Patient's Weight                    
(0012, 0062) Patient Identity Removed            
(0008, 0050) Accession Number
(0008, 0021) Series Date
```

Some of these tags will result in a pydicom object rather than a Python base class:

* (0010, 0010) Patient's Name - returns a [pydicom.valuerep.PersonName](https://pydicom.github.io/pydicom/stable/reference/generated/pydicom.valuerep.PersonName.html)
    (this can be cast to a string)
* (0010, 1030) Patient's Weight - returns a [pydicom.valuerep.DSfloat](https://pydicom.github.io/pydicom/dev/reference/generated/pydicom.valuerep.DSfloat.html)
    (this can be cast to a float)
    
Some other values (like dates and ages) will be represented as strings, but are better loaded as a different type.

If necessary, reference the documentation and select an appropriate method to get the data in the format you desire.

In [2]:
# your code here

from glob import glob

# there are a number of ways to build a pandas dataframe from scratch.  This method builds a list of lists,
# then converts that into a dataframe.
list_data = []
for filename in glob('../resources/**/*.dcm', recursive=True):
    img = pydicom.dcmread(filename)
    list_data.append([
        str(img[0x00100010].value),
        img[0x00100020].value,
        img[0x00100030].value,
        img[0x00100040].value,
        int(img[0x00101010].value.replace('Y', '')),
        float(img[0x00101030].value),
        img[0x00120062].value,
        img[0x00080050].value,
        pd.to_datetime(img[0x00080021].value, format='%Y%m%d')
    ])
    
data = pd.DataFrame(list_data, 
                    columns=['pt_name', 'pt_id', 'pt_birth_date', 'pt_sex', 'pt_age', 'pt_weight', 'pt_deid',
                            'acc_num', 'series_date'])

print(data.dtypes)
display(data)

pt_name                  object
pt_id                    object
pt_birth_date            object
pt_sex                   object
pt_age                    int64
pt_weight               float64
pt_deid                  object
acc_num                  object
series_date      datetime64[ns]
dtype: object


,pt_name,pt_id,pt_birth_date,pt_sex,pt_age,pt_weight,pt_deid,acc_num,series_date
0,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
1,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
2,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
3,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
4,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
5,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
6,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
7,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
8,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02
9,PGBM-001,PGBM-001,,M,52,170.550753,YES,5686274134839343,1992-04-02


## Has every image been deidentified?
One of the first things you should check if working with potential PHI is if the data have been de-identified.  If it has not and you do not have legal access to the PHI you should stop immediately.

The DICOM format has a tag for this, noted above.

In [3]:
# your code here

data[data['pt_deid'] != 'YES']['pt_id'].count()

0

## How many images for each subject and series?
For each subject, print the number of images per subject and series date.

In [4]:
# your code here

data.groupby(by=['pt_id', 'series_date'])[['acc_num']].count() # grouping by patient id and series date, and then counting just the accession number column

acc_num
pt_id    series_date         
PGBM-001 1991-11-19        25
         1992-04-02        24
PGBM-002 1996-08-13        22
         1997-01-17        22
PGBM-003 1995-03-29        22
         1995-10-17        24
PGBM-004 1993-06-22        22
         1994-01-12        46
PGBM-005 1991-07-02        24
         1992-05-05        22

## How many of each sex in the study?

Count the number of subjects by sex.

In [5]:
# your code here

# remember we have one row per image, so we first need just one row per subject, then group by sex and count

data[['pt_id', 'pt_sex']].drop_duplicates().groupby(by='pt_sex').count()

,pt_id
pt_sex,
M,5


## What was the minimum, maximum, and average age of the subjects during their last series?

Find the last scan series by date for each subject, then find the mean age of all subjects from this subset.

hint: look at the [idxmax()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.idxmax.html) method in pandas.

In [6]:
# your code here

# idxmax will give the first encountered index of the maximum for each group.  
# We use that result to get the detailed rows from the data, then perform the aggregate functions on patient age
data.loc[data.groupby('pt_id')['series_date'].idxmax()]['pt_age'].agg(['min', 'max', 'mean'])

min     41.0
max     72.0
mean    53.2
Name: pt_age, dtype: float64

## What was the minimum, maximum, and average weight of the subjects during their last series? 

Similar to the above question, but for weight.

In [7]:
# your code here

data.loc[data.groupby('pt_id')['series_date'].idxmax()]['pt_weight'].agg(['min', 'max', 'mean'])

min      67.130000
max     170.550753
mean    106.944450
Name: pt_weight, dtype: float64

## What was the mean weight loss/gain between the first and the last series?

hint: is there an idxmin()?

In [8]:
# your code here

# using idxmin and idxmax to get the first and last series rows.  index by pt_id so pandas can match the rows easily
first = data.loc[data.groupby('pt_id')['series_date'].idxmin()].set_index('pt_id')['pt_weight']
last = data.loc[data.groupby('pt_id')['series_date'].idxmax()].set_index('pt_id')['pt_weight']

display(first)
display(last)

# subtract the first from the last, so weight loss is negative and weight gain is positive
display(last - first)

display((last - first).mean())

pt_id
PGBM-001    174.633085
PGBM-002     80.740000
PGBM-003    109.660505
PGBM-004     86.640000
PGBM-005    114.305292
Name: pt_weight, dtype: float64

pt_id
PGBM-001    170.550753
PGBM-002     78.020000
PGBM-003    111.130000
PGBM-004     67.130000
PGBM-005    107.891495
Name: pt_weight, dtype: float64

pt_id
PGBM-001    -4.082332
PGBM-002    -2.720000
PGBM-003     1.469495
PGBM-004   -19.510000
PGBM-005    -6.413797
Name: pt_weight, dtype: float64

-6.251326819545997